In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
import pandas as pd

# Sample dataset
data = {
    "text": [
        "Limited offer! Only 2 left in stock!",
        "Subscribe now and cancel anytime!",
        "You must enter your email to continue.",
        "Best deals only for VIP members.",
        "Free trial—No credit card required.",
        "Click here to unsubscribe (hard to find link).",
        "By signing up, you agree to our 20-page terms & conditions.",
        "Your cart will expire in 5 minutes!",
        "No hidden fees or surprise charges.",
        "Easily manage your subscription settings."
    ],
    "label": [1, 0, 1, 1, 0, 1, 1, 1, 0, 0]  # 1 = Dark pattern, 0 = No dark pattern
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("dark_patterns_dataset.csv", index=False)

print("Dataset saved as 'dark_patterns_dataset.csv'")


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


In [ ]:
X = list(df["text"])
y = list(df["label"])


In [ ]:
X_tokenized = tokenizer(X, truncation=True, padding=True, max_length=20, return_tensors="tf")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert input_ids to a NumPy array
X_array = np.array(X_tokenized["input_ids"])

# Now perform the split
X_train, X_test, y_train, y_test = train_test_split(X_array, y, test_size=0.2, random_state=42)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16)


In [ ]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, create_optimizer
from tensorflow.keras.optimizers import Adam  # Correct import

# Load the model (ensure from_pt=True if loading from a PyTorch model)
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, from_pt=True)

# Define optimizer with Hugging Face's create_optimizer
num_train_steps = len(train_dataset) # Replace with your actual number of training steps
optimizer, lr_schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
)

# Compile the model 
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


In [ ]:
model.save("dark_pattern_model")
